In [1]:
from qdrant_client import QdrantClient, models
import pandas as pd
import numpy as np
import os
import math

In [2]:
basePath = os.path.abspath('')
basePath

'C:\\Users\\gorku\\Documents\\bmstu\\RAG24\\late-chunking-ru'

### Первым делом узнаем лидера среди документов по количеству чанков, чтобы гарантировать правильный выбор количества чанков при поиске (это документ из 58 чанков)

In [3]:
%%time
last_idx = 0
for i in range(22):
    chunks_embedded_n = pd.read_pickle(basePath + "\\ai-forever-ria-news-retrieval\\chunks_embedded_"+str(i)+".pkl")
    values, counts = np.unique(chunks_embedded_n['doc_id'], return_counts=True)
    ind = np.argmax(counts)
    print(f"doc_id {values[ind]} count is {counts[ind]}")
    del chunks_embedded_n

doc_id 819 count is 42
doc_id 42280 count is 50
doc_id 66659 count is 58
doc_id 126237 count is 36
doc_id 134092 count is 32
doc_id 163645 count is 48
doc_id 197224 count is 36
doc_id 230948 count is 30
doc_id 284447 count is 32
doc_id 310520 count is 54
doc_id 341400 count is 31
doc_id 361347 count is 43
doc_id 393814 count is 36
doc_id 420280 count is 27
doc_id 448765 count is 41
doc_id 499702 count is 49
doc_id 513972 count is 25
doc_id 563343 count is 40
doc_id 578147 count is 25
doc_id 627485 count is 37
doc_id 657075 count is 33
doc_id 701816 count is 30
CPU times: total: 3min
Wall time: 4min 58s


### Итак, чтобы гарантировать верность рассчёта метрик поиска по документам при k=5, гарантированно достаточно искать 5*58=290 чанков при отбрасывании повторных значений документа-источника чанка

In [4]:
client = QdrantClient(url="http://localhost:6333", prefer_grpc=True)
client

In [5]:
client.get_collection(collection_name="{chunks_embedded_trad}")

CollectionInfo(status=<CollectionStatus.GREEN: 'green'>, optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>, vectors_count=None, indexed_vectors_count=5518911, points_count=5518911, segments_count=17, config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=1024, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=True, datatype=None, multivector_config=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=None), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_a

In [6]:
client.get_collection(collection_name="{chunks_embedded_new}")

CollectionInfo(status=<CollectionStatus.GREEN: 'green'>, optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>, vectors_count=None, indexed_vectors_count=5518911, points_count=5518911, segments_count=16, config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=1024, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=True, datatype=None, multivector_config=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=None), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_a

In [7]:
queries_embedded = pd.read_pickle(basePath + "\\ai-forever-ria-news-retrieval\\queries_embedded.pkl")
test = pd.read_json(basePath + "\\ai-forever-ria-news-retrieval\\test.jsonl", lines=True)
del test['score']

In [8]:
test.head()

,query-id,corpus-id
0,0,670487
1,1,116739
2,2,26225
3,3,288389
4,4,256787


In [9]:
queries_embedded.head()

,_id,text,embedding
0,0,сми: планируется создание в рф несетевых магаз...,"[-0.02244736, -0.028930796, 0.015104426, -0.03..."
1,1,"барулин сожалеет, что набокова нет на хоккейно...","[0.07624497, -0.14052229, -0.10188488, 0.03020..."
2,2,"британские куры забили насмерть лису, проникшу...","[-0.026123617, -0.019093627, 0.045364596, 0.03..."
3,3,путин: сотрудники овд получат выплаты на приоб...,"[-0.050970174, -0.03160782, 0.018168373, 0.026..."
4,4,"футболист ""зенита"" данни пропустит стыковые иг...","[0.017085163, -0.15103693, -0.11232578, -0.019..."


In [10]:
for i, row in test.iterrows():
    if row['query-id'] != i:
        print("Error 1!")

for i, row in queries_embedded.iterrows():
    if row['_id'] != i:
        print("Error 2!")

In [11]:
def ordered_unique(lst):
    p = np.unique(lst, return_index=True)
    return p[0][np.argsort(p[1])]

def get_top_k_docs_by_id(k, id, collections, chunck_k):
    q = queries_embedded.iloc[id]['embedding']
    return [ordered_unique([i.payload['doc_id'] for i in client.query_points(
                collection_name=collection,
                query=q,
                limit=chunck_k
            ).points])[:k] for collection in collections]

def get_true_rel_by_id(id):
    return test.iloc[id]['corpus-id']

def get_top_k_rel_by_id(k, id, collections, chunck_k):
    true_search_result = get_true_rel_by_id(id)
    tops = get_top_k_docs_by_id(k, id, collections, chunck_k)
    return [(top == true_search_result).astype(int) for top in tops]

def DCG_K(rel, k):
    res = 0
    for i in range(1, k + 1):
        res += rel[i-1] / math.log2(1 + i)
    return res

def NDCG_K(rel, k):
    dcg = DCG_K(rel, k)
    rel_sorted = rel.copy();
    rel_sorted[::-1].sort()
    idcg = DCG_K(rel_sorted, k)
    if idcg == 0:
        return 0
    return dcg / idcg

def AP_K(rel, k):
    res = 0
    n = 0
    for i in range(1, k + 1):
        n += rel[i-1]
        res += n * rel[i-1] / i
    if n == 0:
        return 0
    return res / n

In [12]:
k = 5
chunck_k = 290
collections = ["{chunks_embedded_trad}", "{chunks_embedded_new}"]

In [14]:
%%time
ndcg_sum = [0] * len(collections)
ap_sum = [0] * len(collections)
test_size = test.shape[0]
for i in range(test_size):
    rel = get_top_k_rel_by_id(k, i, collections, chunck_k)
    for j in range(len(collections)):
        ndcg_sum[j] += NDCG_K(rel[j], k)
        ap_sum[j] += AP_K(rel[j], k)

for j in range(len(collections)):
    print(f"{collections[j]}\n\tMNDCG_K={ndcg_sum[j] / test_size}\n\tMAP_K={ap_sum[j] / test_size}")

{chunks_embedded_trad}
	MNDCG_K=0.6870302533259881
	MAP_K=0.6542466666666595
{chunks_embedded_new}
	MNDCG_K=0.8500745753240168
	MAP_K=0.8270649999999947
CPU times: total: 3min 58s
Wall time: 8h 4min 52s


In [15]:
client.close()